In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# IMPORTING LIBRARIES

In [ ]:
pip install bs4

In [ ]:

import random

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
lemmatizer = WordNetLemmatizer()
nltk.download("stopwords") 
from nltk.corpus import stopwords
nltk.download("words")

import re
from bs4 import BeautifulSoup          #To remove html tags while preprocessing reviews.         #To remove html tags while preprocessing reviews.

# EXTRACTING DATA 

In [ ]:
data = pd.read_csv(r"/kaggle/input/twitter-sentiment-analysis-hatred-speech/train.csv")
data.isnull().sum()

In [ ]:
tweets=data["tweet"].values.tolist()
labels=data["label"].values

In [ ]:
print(tweets[:10])
print(len(tweets))
print(labels[:10])
print(len(data[data["label"]==0]))
print(len(data[data["label"]==1]))

In [ ]:
# PRE-PROCESSING:REMOVING STOPWORDS,NON-ENGLISH WORDS AND LEMMATIZING AND FILTERING WORDS WITH LENGTH LESS THAN 3.

def remove_underscores(sentence):
    sentence= sentence.replace("_"," ")
    return sentence

stop_words = set(stopwords.words('english'))
english_words = set(nltk.corpus.words.words())

def remove_extra_words(sentence):     #remove stop words and meaningless words
    new_sentence=""
    for w in sentence.split():
        w=w.lower()
        if w in english_words and w not in stop_words and w.isalpha():
            new_sentence=new_sentence+" "+w
    return new_sentence

def lemmatize_and_filter(sentence, min_word_length):        #to lemmatize words and lose the ones with length less than equal to 3.
    sent = ""
    for word in sentence.split():
        word=word.lower()
        if len(lemmatizer.lemmatize(word))>3:
            sent= sent+" "+lemmatizer.lemmatize(word)
    return(sent)

i.EXTRACTING THE REVIEWS IN STRING TYPE.

ii.REMOVING LABELS, NUMBERS, URLs,HTML TAGS AND PUNCTUATIONS.

iii.TRANSFORMING ALL REVIEWS TO LOWER CASE.

iv. REMOVING STOPWORDS.

v. LEMMATIZING WORDS.

In [ ]:
#re.sub(r'[^\w\s],"",string) to remove punctuations.
#re.sub(r'^https?:\/\/.*[\r\n]*', '', string) to remove urls.
#strip() to remove whitespaces
#BeautifulSoup(string, "lxml").text) to remove html tags





def preprocessed_data(data):        #takes in list of raw data and returns list of processed string sentences and the list of their corresponding labels.
    reviews = [lemmatize_and_filter(remove_extra_words(BeautifulSoup(re.sub(r'[^\w\s]|^https?:\/\/.*[\r\n]*|\d+', '', remove_underscores(str(lines)).strip().lower()), "lxml").text),3) for lines in data]
    return(reviews)

In [ ]:
#  **TRAIN-VALIDATION SPLIT**

def split_train_into_tain_validate(data_,validation_ratio):
    
    data=data_
    
    train_size = int(len(data)*(1-validation_ratio))
    
    random.shuffle(data)

    validate_samples= data[train_size:]

    data = data[:train_size]
    
    return(data,validate_samples)

In [ ]:
# **CLASSIFICATION [POSITIVE OR NEGATIVE REVIEW-> SENTIMENT ANALYSIS]**

## TENSORFLOW (PREPROCESSING) : CREATING THE DATA TO TRAIN-TEST THE CLASSIFIER.

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
training_sentences=preprocessed_data(tweets)

In [ ]:
tokenizer = Tokenizer(oov_token="<OOV>")           #OOV tag used for unfamiliar words encountered by the model
tokenizer.fit_on_texts(training_sentences)


In [ ]:
word_index = tokenizer.word_index
word_index #returns a dictionary with key= words, values= tokens(numbers)

In [ ]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences)

del training_sentences,training_sequences

# Need this block to get it to work with TensorFlow 

training_padded = np.array(training_padded)
#labels

Artificial Neural Network

In [ ]:
vocab_size = max(list(tokenizer.word_index.values()))+1

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 10),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

In [ ]:
cb = [ tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=.25, patience=2, cooldown=5)]
history = model.fit(training_padded, labels, epochs=50, validation_split=0.2,verbose=1,callbacks = cb)

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

ANN with different optimizer and without callbacks.

In [ ]:
vocab_size = max(list(tokenizer.word_index.values()))+1

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 10),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')])

model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

model.summary()

In [ ]:

history = model.fit(training_padded, labels, epochs=50, validation_split=0.2,verbose=1)

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

Conclusion: The callbacks were stoping the model from learning. Though the ideal model would have decrease in the graph plotted for loss and increase in the graph for accuracy with epochs increasing.

Recurrent Neural Network

In [ ]:
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Embedding(vocab_size, 100))
model2.add(tf.keras.layers.Dropout(0.5))
model2.add(tf.keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model2.add(tf.keras.layers.Dense(1, activation='relu'))

model2.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

model2.summary()

In [ ]:
callbacks = [ tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=.25, patience=2, cooldown=0)]

history2 = model2.fit(training_padded, labels, epochs=40,
                    batch_size=500,
                    validation_split=0.25,
                    verbose=1,callbacks=callbacks)  

In [ ]:
plot_graphs(history2, "accuracy")
plot_graphs(history2, "loss")

Hence, this model acts most idealy compared to the other ANN models.

Checking with test data.

In [ ]:
data = pd.read_csv("/kaggle/input/twitter-sentiment-analysis-hatred-speech/test.csv")
data

In [ ]:
tweets=data["tweet"].values.tolist()
tweets

In [ ]:
test_sentences=preprocessed_data(tweets)
tokenizer = Tokenizer(oov_token="<OOV>")           #OOV tag used for unfamiliar words encountered by the model
tokenizer.fit_on_texts(test_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences)

del test_sentences,test_sequences
test_padded = np.array(test_padded)

model2.predict_classes(test_padded)
